[ja]: #
# Chapter 2. NumPy基礎

[en]: #
# Chapter 2. Basics of NumPy

[ja]: #
この章ではPythonで高速に数値計算するためのオープンソースライブラリである[NumPy](https://numpy.org/)の基本的な使い方を学びます。
NumPyの計算は型付きの多次元配列 (テンソルや行列)上で行われ、同時にそれを操作するためのいくつかの基本的な数学関数が提供されています。
またその汎用性や性能の高さからSciPyやmatplotlibといったほかのPythonの主要ライブラリの基盤になっておりとても重要なライブラリです。

[en]: #
This chapter introduces the basics of [NumPy](https://numpy.org/), the standard open-source library for efficient numerical computing in Python.
NumPy provides typed multidimensional arrays along with a comprehensive collection of numerical routines, forming the foundation of many scientific Python libraries such as SciPy and Matplotlib.

[ja]: #
## 前書き

[en]: #
## Introduction

[ja]: #
まずNumPyを使用する利点と、この章の全体構成を説明します。
いくつか細かい話を含みますので、演習に移りたい方はスキップして次の節から始めてください。

[en]: #
This section summarizes the advantages of using NumPy and outlines the chapter structure.
It includes technical background; if you prefer to proceed directly to the practical exercises, you may skip these details.

[ja]: #
### なぜNumPyか？

[en]: #
### Why NumPy?

[ja]: #
そもそもNumPyを使用する主要な理由として、Pythonそのものの遅さが挙げられます。
Pythonはインタプリタ言語 (コードを一行ずつ解釈し実行する言語)で柔軟にコードを変更できます。
一方でコンパイル言語(コード全体をあらかじめ機械語に翻訳する言語)であるCやJavaと比べるとその速度は相当に遅いです。
例えば[素数を列挙するベンチマーク](https://github.com/kostya/benchmarks/tree/master?tab=readme-ov-file#primes)ではC++に比べ30倍以上時間がかかっており、実際一般にPythonそれ自体は遅い言語に分類される場合が多いです。

ではPythonはなぜ遅いのでしょうか？
[コンパイラ最適化](https://ja.wikipedia.org/wiki/%E3%82%B3%E3%83%B3%E3%83%91%E3%82%A4%E3%83%A9%E6%9C%80%E9%81%A9%E5%8C%96) 等の恩恵が受けられない等のインタプリタ言語に由来する要因もありますが、一番はPythonが動的型付け言語、つまり型を逐次チェックが要求される点が影響していると考えられます。
例えば`a + b`といった単純な足し算でも、以下の処理がPythonでは行われています。
1. `a`が足し算(`+`)をサポートしているかチェック
2. `a`の足し算を定義する関数を取得
3. `b` が足し算に適切なデータかチェック
4. `a`と`b`の値を取り出し和をとる

このように演算 (この場合`+`)のたびに、逐一型のチェックと対応する関数の呼び出しを行う必要があり、それが大きなオーバーヘッドとなっています。
対照的に、CやJava等の静的型付け言語では、そもそも未定義の演算はコンパイル時にエラーとなるため、1~3の部分が実行時には不要になり結果的に高速な演算ができます。

NumPyは言い換えると、そうした静的型付け言語による速度面でのメリットを、動的型付け言語であるPythonでも享受するために開発されたライブラリです。
NumPyは計算コストの高い箇所をC (と一部C++とFORTRAN)で実装されている他、LAPACK/BLASといった数値線形代数ライブラリを使用しており非常に高速に動作します ([行列積のベンチマーク](https://github.com/kostya/benchmarks/tree/master?tab=readme-ov-file#matmul)参照)。

[en]: #
NumPy addresses performance limitations of Python that make many scientific computations impractical in pure Python.

Python is an interpreted language, which enables for rapid development and flexibility.
However, compared with compiled languages such as C or Java (whose code is translated to machine code in advance), Python is substantially slower for many numerical workloads.
For example, [a benchmark enumerating prime numbers](https://github.com/kostya/benchmarks/tree/master?tab=readme-ov-file#primes) shows Python can be more than 30× slower than optimized C++ implementations (see the linked benchmark).

Why is Python relatively slow?
Several factors contribute: it can't exploit compiler optimizations the way compiled languages can, and its dynamic typing requires runtime type inspection and method dispatch.
For instance, even a simple expression like `a + b` typically involves:

1. Checking whether a implements the operation `+`.
2. Looking up the function that performs addition for `a`.
3. Verifying that `b` is compatible with that operation.
4. Extracting the values and computing the sum.

These runtime checks and dispatch steps add measurable overhead.
By contrast, statically typed languages resolve such operations at compile time, avoiding much of this cost and enabling faster execution.

NumPy allows Python programs to take advantage of the performance benefits of statically typed, compiled code.
Its computational kernels are implemented in C (with parts written in C++ and Fortran), and it relies on optimized linear-algebra libraries such as BLAS and LAPACK. As a result, operations such as matrix multiplication achieve high performance.

[ja]: #
### なぜ静的型付け言語を使わないのか？

[en]: #
### Why not use a statically typed language?

[ja]: #
ここまで読まれた方の中で、「Pythonではなくそもそも初めからCやJava等で数値計算を行えばよいのではないか？」と思われた方もいらっしゃるかもしれません。
これは尤な疑問ですが、結論から先に申し上げると、特に初心者の方にはあまりおすすめできません。
というのも高性能計算の非専門家では太刀打ちできないくらいに、あまりにNumPyをはじめとした高性能計算のライブラリが既に洗練されているからです。
例えば単純な先ほど引用した[行列積のベンチマーク](https://github.com/kostya/benchmarks/tree/master?tab=readme-ov-file#matmul)では、[三重ループをそのまま実装したCのコード](https://github.com/kostya/benchmarks/blob/master/matmul/matmul.c)が、[NumPyの`np.dot`を使用したコード](https://github.com/kostya/benchmarks/blob/master/matmul/matmul-numpy.py)と比較して50倍程度の時間がかかっています。
このように基本的な機能の実装においては、専門家でない限り自前での最適化の試みが無意味どころか逆効果になります。
したがって基本的にはその労力をほかの点に費やすべきともいえます (Cf. [四角い車輪の再発明](https://en.wikipedia.org/wiki/Reinventing_the_wheel#Related_phrases))。
実際AI・機械学習の分野では近年 (2010年代以降)、数値計算の細部はライブラリ (と高性能計算の専門家)に任せ、(Pythonの柔軟さを活用して) 新しいアイディアの実装や実証をその上で次々に行う研究・開発スタイルが主流となっています。

[en]: #
One might wonder why numerical code is not simply written directly in C or another statically typed language.
For most practitioners, especially beginners, that is not the most productive choice.
Libraries such as NumPy already incorporate extensive optimizations and domain expertise.
Reimplementing these features without comparable expertise is rarely efficient.
For example, a naive C implementation of [matrix multiplication](https://github.com/kostya/benchmarks/tree/master?tab=readme-ov-file#matmul) (three nested loops) can be orders of magnitude slower than a tuned library call such as NumPy's `np.dot`, due to missing algorithmic and low-level optimizations.
Consequently, researchers and engineers typically focus on higher-level problems rather than low-level optimization tasks (see the [*reinventing the wheel*](https://en.wikipedia.org/wiki/Reinventing_the_wheel#Related_phrases) article).
In recent years, the AI and machine-learning communities have standardized on delegating low-level numerical computation to optimized libraries while using Python's flexibility for rapid prototyping and experimentation.

[ja]: #
### この章で扱われる内容

[en]: #
### What this chapter covers

[ja]: #
まず具体的な行列・テンソルの構成を通して、多次元配列の作成・操作の方法を学習します。
次に配列のインデクシング (e.g., `a[idx]`)と、論理ならびに要素の検索方法を学びます。
三つ目に、数学、統計に関連する関数を扱います。
最後に特にこの教材全体を通して重要となる線形代数のトピックを扱います。

[en]: #
This chapter begins with how to construct and manipulate multidimensional arrays (matrices and tensors).
It then introduces array indexing, logical operations, elementwise selection, and search techniques such as `a[idx]`.
We also review commonly used mathematical and statistical routines, followed by essential linear-algebra tools that appear throughout this RC bootcamp.

[ja]: #
### NumPyを使うときのコツ

[en]: #
### Tips for using NumPy

[ja]: #
最後に演習に移る前に役に立つNumPyを用いたコーディングのコツを2点記します。
1. `for`の使用をできるだけ回避する
2. 要素の型に注意を払う

まず`for`はなるべく使用しないでください。
先述のとおり、Pythonでは演算の度に型検証に関するオーバーヘッドが発生します。
したがって`for`文では繰り返しの回数の分だけ遅くなります。
代わりにできるだけNumPyの配列上で完了させてください。
実際この節ではほとんどのケースで`for`を用いないでの実装が可能です。

次に要素の型を意識してください。
NumPyが高速なのは型を全部そろえて先述のオーバーヘッドをなくしているからです。
逆にQ1.1.で扱うとおり、NumPyでサポートされていない型を使用すると一気に速度が低下します。
これらの点を意識すると効率的な数値計算が可能となります。

[en]: #
Lastly, here are two tips for coding with NumPy before we move on to the exercise:
1. Avoid using `for` loops whenever possible.
2. Be mindful of array element types.

First, try to avoid `for` loops.
Each Python-level operation incurs overhead due to dynamic type checking.
Therefore, using `for` loops slows down the process proportionally to the number of iterations.
Instead, strive to complete operations on NumPy arrays whenever possible.
In fact, most problems in the exercise section can be solved without resorting to `for` loops.

Next, pay attention to the types of array elements.
NumPy achieves high performance by ensuring that all elements share the same fixed data type.
Using unsupported or mixed types can significantly degrade performance.
By keeping these points in mind, you can perform efficient numerical computations.

[ja]: #
### 準備

[en]: #
### Preparation

[ja]: #
前節同様、まず次のセルを実行してください。

[en]: #
As in the previous chapter, begin by running the initialization cell below.

In [ ]:
import sys

import numpy as np

if "google.colab" in sys.modules:
    from google.colab import drive  # type: ignore

    if False:  # Set to True if you want to use Google Drive and save your work there.
        drive.mount("/content/gdrive")
        %cd /content/gdrive/My Drive/[[PROJECT_NAME]]/
        # NOTE: Change it to your own path if you put the zip file elsewhere.
        # e.g., %cd /content/gdrive/My Drive/[PATH_TO_EXTRACT]/[[PROJECT_NAME]]/
    else:
        pass
        %cd /content/
        !git clone --branch [[BRANCH_NAME]] https://github.com/rc-bootcamp/[[PROJECT_NAME]].git
        %cd /content/[[PROJECT_NAME]]/
else:
    sys.path.append(".")

from utils.tester import load_from_chapter_name

test_func, show_solution = load_from_chapter_name("02_numpy_basics")

[ja]: #
## 1. 配列の作成

[en]: #
## 1. Constructing arrays

[ja]: #
まず配列を作成してみましょう。
基本的には`np.array`を使用するとNumPy上で配列を作成できます。
また0行列、1行列はそれぞれ`np.zeros`・`np.ones`で作成できます。
空白のセルでいろいろ試してみてから問題に移りましょう。

[en]: #
We begin with array creation.
Use `np.array` to construct arrays from Python sequences; for filled arrays, use `np.zeros` or `np.ones`.
Experiment interactively in a blank cell before attempting the exercises.

Q1.1.

[ja]: #
整数からなるリスト $A$ が与えられる。
$A$は非常に大きな整数を含む可能性があり、`np.array`を用いて`np.ndarray`型にそのまま変換すると、例えば以下のように要素が`object`型として扱われてしまう。
```py
In [*]: np.array([10**20])
Out[*]: array([100000000000000000000], dtype=object)
```
`object`型は`numpy`上でポインタ (メモリアドレスを保持する変数)をのみ保持し、計算そのものはPython上(*not* `numpy`)で行われるため、`numpy`上で計算が完了する場合と比べて以下のようにかなり時間がかかってしまう。
```py
In [*]: x_int = np.arange(100000, dtype=np.int64)
   ...: x_obj = np.arange(100000, dtype='object')
   ...: %timeit x_int * x_int
   ...: %timeit x_obj * x_obj
54.3 µs ± 189 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
1.45 ms ± 9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
```
そこで要素を`numpy`でサポートされている型に変換したいが、`np.int64`でキャストしてしまうと64-bit符号付き整数の最大値 (`np.iinfo(np.int64).max`で得られる)である`9223372036854775807` ( $2^{63} - 1$ )を超えてしまうため、オーバーフローが発生する。
```py
In [*]: np.array([10**20], dtype=np.int64)
---------------------------------------------------------------------------
OverflowError                             Traceback (most recent call last)
Cell In[*], line 1
----> 1 np.array([10**20], dtype=np.int64)

OverflowError: Python int too large to convert to C long
```
これを防ぐため、リストに含まれる整数の範囲に応じて型を変えたい。
$A$ の最小・最大要素を $a_\text{min}, a_\text{max}$ とした時、 $-2^{63} \leq a_\text{min} \leq a_\text{max} \leq 2^{63}-1$ なら`np.int64`、それ以外なら64-bit浮動小数点型`np.float64`に変換された要素を有する`np.ndarray`に変換するコードを書け。

[en]: #

A list of integers $A$ is given.
The list $A$ may contain very large integers.
If you convert it directly to an `np.ndarray` type using the `np.array` function, the resulting array may have the `object` dtype, as shown below:
```py
In [*]: np.array([10**20])
Out[*]: array([100000000000000000000], dtype=object)
```
The `object` type in `numpy` only holds pointers (variables that store memory addresses), and computations are performed on Python itself (*not* `numpy`), which takes considerably more computation time.
For example:
```py
In [*]: x_int = np.arange(100000, dtype=np.int64)
   ...: x_obj = np.arange(100000, dtype='object')
   ...: %timeit x_int * x_int
   ...: %timeit x_obj * x_obj
54.3 µs ± 189 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
1.45 ms ± 9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
```
Therefore, we want to convert the elements to a data type supported by `numpy`, but casting with `np.int64` can lead to overflow if the values exceed the maximum value of a 64-bit signed integer ($2^{63} - 1$ or `9223372036854775807`, as obtained with `np.iinfo(np.int64).max`).
```py
In [*]: np.array([10**20], dtype=np.int64)
---------------------------------------------------------------------------
OverflowError                             Traceback (most recent call last)
Cell In[*], line 1
----> 1 np.array([10**20], dtype=np.int64)

OverflowError: Python int too large to convert to C long
```
To prevent this, we want to change the data type according to the range of integers in the list $A$.
If the minimum and maximum elements of $A$, denoted as $a_\text{min}$ and $a_\text{max}$ respectively, satisfy the conditions $-2^{63} \leq a_\text{min} \leq a_\text{max} \leq 2^{63}-1$, then $A$ should be converted to an `np.ndarray` with elements of type `np.int64`.
Otherwise, it should be converted to an `np.ndarray` with elements of type `np.float64`.
Write code that achieves such process.

[END]: #
- $A$: `list` of `int`
- $1 \leq |A| \leq 10^{4}$
- $0 \leq |a_\text{min}|, |a_\text{max}| \leq 10^{100}$
- Return: `np.ndarray`
- Sample
  - `[2**100, 10]`->`array([1.2676506e+30, 1.0000000e+01])`
  - `[2**60, 10]`->`array([1152921504606846976, 10])`
  - `[-2**63, 2**63 - 1]`->`array([-9223372036854775808,  9223372036854775807])`
  - `[-2**63, 2**63]`->`array([-9.22337204e+18,  9.22337204e+18])`

[tips]: #
- [`np.array`](https://numpy.org/doc/stable/reference/generated/numpy.array.html)
- [`np.asarray`](https://numpy.org/doc/stable/reference/generated/numpy.asarray.html)
- [`np.dtype`](https://numpy.org/doc/stable/reference/generated/numpy.dtype.html)
- [`np.iinfo`](https://numpy.org/doc/stable/reference/generated/numpy.iinfo.html)

[/tips]: #

In [ ]:
def solution(arr):
    # BEGIN
    vmax, vmin = max(arr), min(arr)
    if np.iinfo(np.int64).min <= vmin <= vmax <= np.iinfo(np.int64).max:
        return np.array(arr, dtype=np.int64)
    else:
        return np.array(arr, dtype=np.float64)
    # END


test_func(solution, "01_01")
# show_solution("01_01")  # Uncomment it to see the solution.

Q1.2.

[ja]: #
整数 $a, d, n$ が与えらえる。
$a_{0}=a, a_{k+1}=a_{k}+d$ となる等差数列 $a_0,~\ldots,~a_{n-1}$を要素として有する`np.ndarray`型の配列を出力するコードを書け。

[en]: #
Some integers $a, d, n$ are given.
Write code to output an `np.ndarray` of length $n$ containing the arithmetic sequence $a_0,~\ldots,~a_{n-1}$, following the rule $a_{0}=a, a_{k+1}=a_{k}+d$.

[END]: #
- $a, d, n$: `int`
- $0 \leq |a|, |d| \leq 10^{4}$
- $1 \leq n \leq 10^{4}$
- Return: `np.ndarray`
  - `shape`: `(n,)`
  - `dtype`: `np.int64`
- Sample
  - `5, 3, 4`->`array([5, 8, 11, 14])`
  - `4, -2, 3`->`array([4, 2, 0])`

[tips]: #
- [`np.arange`](https://numpy.org/doc/stable/reference/generated/numpy.arange.html)

[/tips]: #

In [ ]:
def solution(a, d, n):
    # BEGIN
    return np.arange(a, a + d * n, d)
    # END


test_func(solution, "01_02")
# show_solution("01_02")  # Uncomment it to see the solution.

Q1.3.

[ja]: #
非負の実数 $a, b$ と整数 $n$ が与えられる。
$a_{0}=a,a_{n-1}=b$ となる等比数列$a_0,\ldots a_{n-1}$ を要素として有する要素数 $n$ の`np.ndarray`型の配列を出力するコードを書け。

[en]: #
Given non-negative real numbers $a, b$ and an integer $n$, write code to output an `np.ndarray` of length $n$ containing the geometric sequence $a_0,~\ldots,~a_{n-1}$, satisfying $a_{0}=a, a_{n-1}=b$.

[END]: #
- $a, b$: `float`
- $n$: `int`
- $0 \leq a, b \leq 10^{4}$
- $2 \leq n \leq 100$
- Return: `np.ndarray`
  - `shape`: `(n,)`
  - `dtype`: `np.float64`
- Sample
  - `1.0, 8.0, 4`->`array([1., 2., 4., 8.])`
  - `4.0, 1.0, 3`->`array([4., 2., 1.])`
  - `2.0, 2.0, 3`->`array([2., 2., 2.])`

[tips]: #
- [`np.linspace`](https://numpy.org/doc/stable/reference/generated/numpy.linspace.html)
- [`np.logspace`](https://numpy.org/doc/stable/reference/generated/numpy.logspace.html)

[/tips]: #

In [ ]:
def solution(a, b, n):
    # BEGIN
    return a * np.logspace(0, 1, n, base=b / a)
    # return a * ((b / a)  ** np.linspace(0, 1, n))
    # END


test_func(solution, "01_03")
# show_solution("01_03")  # Uncomment it to see the solution.

Q1.4.

[ja]: #
行列 $A\in \mathbb{R}^{m\times n}$ が与えられる。
Aの0行0列目の要素を $a_{00}$ とする時、 $b_{ij}=a_{00}$ となる行列 $B=(b_{ij})_{0\leq i < m, 0\leq j < n}$ を作成するコードを書け。
ただし $A$ と要素の型は揃えよ。

[en]: #
A matrix $A\in \mathbb{R}^{m\times n}$ is given.
Write code that creates a matrix $B=(b_{ij})_{0\leq i < m, 0\leq j < n}$ where each element $b_{ij}$ equals the element at the 0th row and 0th column of $A$ denoted as $a_{00}$, i.e. $b_{ij}=a_{00}$.
Ensure that the data type of $B$ matches that of $A$.

[END]: #
- $A$: `np.ndarray`
  - `shape`: `(m, n)`
  - `dtype`: `np.int64` | `np.float64`
- $1 \leq |m|, |n| \leq 10^{3}$
- $0 \leq |a_{ij}| \leq 10^{4}$
- Return: `np.ndarray`
  - `shape`: `(M, N)`
  - `dtype`: `np.int64` | `np.float64`
- Sample
  - `array([[2., 0., 0.]])`->`array([[2., 2., 2.]])`
  - `array([[4, 3, 2], [2, 3, 4]])`->`array([[4, 4, 4], [4, 4, 4]])`

[tips]: #
- [`np.full_like`](https://numpy.org/doc/stable/reference/generated/numpy.full_like.html)

[/tips]: #

In [ ]:
def solution(arr):
    # BEGIN
    return np.full_like(arr, arr[0, 0])
    # END


test_func(solution, "01_04")
# show_solution("01_04")  # Uncomment it to see the solution.

Q1.5.

[ja]: #
整数 $m,n$ が与えられる。
$a_{ij}= k~(i + j \equiv k~(\text{mod} 3), 0 \leq k \leq 2) $となるような、「3色のチェッカーボード」状の行列 $A=(a_{ij})_{0 \leq i < M, 0 \leq j < N}$ を作成するコードを書け。

[en]: #
Integers $m,n$ are given.
Write code to create a matrix $A=(a_{ij})_{0 \leq i < M, 0 \leq j < N}$ with a "3-colored checkerboard" pattern, that satisfies $a_{ij}= k~(i + j \equiv k~(\text{mod} 3), 0 \leq k \leq 2)$ .

[END]: #
- $m, n$: `int`
- $1 \leq m, n \leq 10^{3}$
- Return: `np.ndarray`
  - `shape`: `(m, n)`
  - `dtype`: `np.int8`
- Sample
  - `3, 3`->`array([[2, 0, 1],[0, 1, 2], [1, 2, 0]], dtype=int8)`
  - `1, 10`->`array([[2, 0, 1, 2, 0, 1, 2, 0, 1, 2]], dtype=int8)`
  - `5, 1`->`array([[2], [0], [1], [2], [0]], dtype=int8)`

[tips]: #
- [Indexing on ndarrays](https://numpy.org/doc/stable/user/basics.indexing.html)

[/tips]: #

In [ ]:
def solution(m, n):
    # BEGIN
    val = np.zeros((m, n), dtype=np.int8)
    val[2::3, ::3] = 2
    val[1::3, 1::3] = 2
    val[::3, 2::3] = 2
    val[1::3, ::3] = 1
    val[::3, 1::3] = 1
    val[2::3, 2::3] = 1
    return val
    # END


test_func(solution, "01_05")
# show_solution("01_05")  # Uncomment it to see the solution.

Q1.6.

[ja]: #
一桁の整数からなる要素数 $n$ のリスト $A$ が与えられる。
整数 $k~(0\leq k \leq 9)$ に対して、要素数`10`で $k + 1$ 番目が`1`それ以外が`0`となる単位ベクトルを $e^k \in \{0, 1\}^{10}$ とする (このような操作を*one-hot*ベクトル化という)。
$A$の $i$ 番目の要素 $a_{i}$ とする時、各整数を $a_{i}$ を $e^{a_{i}}$ にして並べた行列 $[e^{a_{1}}, e^{a_{2}},~\ldots,~e^{a_{n}}]^\top $を構築するコードを書け。

[en]: #
A list $A$ of length $n$ containing single-digit integers is given.
For an integer $k~(0\leq k \leq 9)$, the *one-hot vector* $e^k \in \{0, 1\}^{10}$ is defined as a vector of length `10` where the $k + 1$-th element is `1` and all other elements are `0`.
Write code that constructs a matrix from $A$, where each integer $a_{i}$ (the $i$-th element of $A$) is replaced with its one-hot vector $e^{a_{i}}$.
In other words, the resulting matrix is $[e^{a_{1}}, e^{a_{2}},~\ldots,~e^{a_{n}}]^\top $.

[END]: #
- $A$: `list` of `int`
- $0 \leq n (=|A|) \leq 10^{3}$
- $0 \leq a_{i} \leq 9$
- Return: `np.ndarray`
  - `shape`: `(..., 10)`
  - `dtype`: `np.int8`
- Sample
  - `[2, 3]`->`array([[0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]], dtype=int8)`

[tips]: #
- [`np.eye`](https://numpy.org/doc/stable/reference/generated/numpy.eye.html)

[/tips]: #

In [ ]:
def solution(arr):
    # BEGIN
    return np.eye(10, dtype=np.int8)[arr]
    # END


test_func(solution, "01_06")
# show_solution("01_06")  # Uncomment it to see the solution.

Q1.7.

[ja]: #
２つの正方対角行列 $A, B \in \mathbb{R}^{n\times n}$ が与えられる (対角行列は $(i, i)$ 要素以外0の行列を指す)。
この時AとBの行列積 $A B$ はアダマール積 $A\odot B$ に一致するが、それを実現するnumpyコード`a * b`は計算量が $O(n^2)$ となり特に大きなサイズの $n$ に対して効率的でない。
$A, B$の対角成分 $a_{ii}, b_{ii}$ のみを抽出し、 $a_{ii}b_{ii}$ を $0\leq i < n$ に対して計算したのち、正方対角行列 $C=(a_{ij} b_{ij})$ を再構築するコードを書け。

[en]: #
2 square diagonal matrices $A, B \in \mathbb{R}^{n\times n}$ are given (a diagonal matrix is defined as a matrix with nonzero elements only at the $(i, i)$ positions).
Under this condition, the matrix product $A B$ coincides with the Hadamard product $A\odot B$, but the numpy code `a * b` for this operation has a time complexity of $O(n^2)$ and is not efficient for large values of $n$.
Write code that extracts the diagonal elements $a_{ii}, b_{ii}$ of $A, B$, computes the product $a_{ii}b_{ii}$ for $0\leq i < n$, and then reconstructs the square diagonal matrix $C=(a_{ij} b_{ij})$.

[END]: #
- $A, B$: `np.ndarray`
  - `dtype`: `np.int64`
  - `shape`: `(n, n)`
- $0 \leq |a_{ii}, b_{ii}| \leq 10^{3}$
- Return: `np.int64`
- Sample
  - `array([[1, 0], [0, 2]]), array([[3, 0], [0, 4]])`->`array([[3, 0], [0, 8]])`

[tips]: #
- [`np.multiply`](https://numpy.org/doc/stable/reference/generated/numpy.multiply.html)
- [`np.diag`](https://numpy.org/doc/stable/reference/generated/numpy.diag.html)

[/tips]: #

In [ ]:
def solution(mat_a, mat_b):
    # BEGIN
    return np.diag(np.diag(mat_a) * np.diag(mat_b))
    # END


test_func(solution, "01_07")
# show_solution("01_07")  # Uncomment it to see the solution.

Q1.8.

[ja]: #
$n$個の整数からなるリスト $A$ ならびに、 $n-1$ 個の整数からなるリスト $B$ が与えらえる。
$A,B$の $i$ 番目の要素を $a_{i},b_{i}$ とする時、以下の式で表現される三重対角行列 $D(\{a_i\},\{b_i\})$ を作成するコードを書け。

[en]: #
A list $A$ containing $n$ integers and a list $B$ containing $n-1$ integers are given.
The $i$-th elements of $A$ and $B$ are denoted as $a_{i}$ and $b_{i}$, respectively.
Write code to create a tridiagonal matrix $D(\{a_i\},\{b_i\})$ represented by the following formula:

[END]: #
$$
\begin{align*}
D(\{a_i\},\{b_i\}) &= \begin{bmatrix}
a_{1} & b_{1} &        &         & \huge{0} \\
b_{1} & a_{2} & b_{2}  & \\
      & b_{2} & a_{3}  & \ddots  & \\
      &       & \ddots & \ddots  & b_{n-1} \\
\huge{0} &    &        & b_{n-1} & a_{n} \\
\end{bmatrix}
.\end{align*}
$$

- $A, B$: `list` of `int`
- $1 \leq |B| < |B| + 1 = |A| \leq 10^{3}$
- $0 \leq |a_i|, |b_i| \leq 10^{3}$
- Return: `np.ndarray`
  - `shape`: `(n, m)`
  - `dtype`: `np.int64`
- Sample
  - `[1, 2, 3], [4, 5]`->`array([[1, 4, 0], [4, 2, 5], [0, 5, 3]])`

[tips]: #
- [`np.diag`](https://numpy.org/doc/stable/reference/generated/numpy.diag.html)

[/tips]: #

In [ ]:
def solution(arr_a, arr_b):
    # BEGIN
    return np.diag(arr_a) + np.diag(arr_b, 1) + np.diag(arr_b, -1)
    # END


test_func(solution, "01_08")
# show_solution("01_08")  # Uncomment it to see the solution.

Q1.9.

[ja]: #
数列$a_{i}$ ($1\leq i \leq n$)に対して以下の形で定義される行列 $C(\{a_{i}\})$ を巡回行列と呼ぶ。

[en]: #
A matrix $C(\{a_{i}\})$ defined in the following manner for a sequence $a_{i}$ ($1\leq i \leq n$) is called a circulant matrix:

[END]: #
$$
\begin{align*}
C(\{a_{i}\})=\begin{bmatrix}
a_1     & a_{n}   & \cdots & a_{3}  & a_{2}   \\
a_2     & a_{1}   & a_{n}    &        & a_{3}   \\
\vdots  & a_{2}   & a_{1}    & \ddots & \vdots  \\
a_{n-1} &         & \ddots   & \ddots & a_{n}   \\
a_{n}   & a_{n-1} & \cdots & a_{2}  & a_{1}   \\
\end{bmatrix}
.\end{align*}
$$

[ja]: #
今2個の整数 $k, n$ が与えられる。

[en]: #
Now, 2 integers $k, n$ are given.
Create code to generate the circulant matrix $C(\{b_{i}\})$ for the sequence $\{b_i\}$ with length $n$ defined by

[END]: #
$$
\begin{align*}
b_{i}=\begin{cases}
1&(i=k)\\
0&(\text{otherwise})
\end{cases}
.\end{align*}
$$

[ja]: #
となる数列 $\{b_i\}$ に対する巡回行列 $C(\{b_{i}\})$ を作成するコードを書け。

[END]: #
- $k, n$: `int`
- $1 \leq k \leq n \leq 10^{3}$
- Return: `np.ndarray`
  - `shape`: `(n, n)`
  - `dtype`: `np.int8`
- Sample
  - `2, 3`->`array([[0, 0, 1], [1, 0, 0], [0, 1, 0]], dtype=np.int8)`

[tips]: #
- [`np.roll`](https://numpy.org/doc/stable/reference/generated/numpy.roll.html)

[/tips]: #

In [ ]:
def solution(k, n):
    # BEGIN
    return np.roll(np.eye(n, dtype=np.int8), k - 1, axis=0)
    # END


test_func(solution, "01_09")
# show_solution("01_09")  # Uncomment it to see the solution.

Q1.10.

[ja]: #
整数 $n~(n\neq 0)$ が与えられる。
$n$ の正負に応じて以下の三角行列 $T(n)$ を作成するコードを書け。

[en]: #
Given a nonzero integer $n~(n\neq 0)$, write code to create a triangular matrix $T(n)$ according to the sign of $n$ as follows:

[END]: #
$$
\begin{align*}
T(n)=\begin{cases}
&
\begin{bmatrix}
1 & 2 & \cdots & n-1 & n \\
  & n + 1 & \cdots & 2n-2 & 2n-1 \\
  &       & \ddots & \vdots & \vdots \\
  & \huge{0} &     & m-2  & m-1 \\
  &       &        &      & m \\
\end{bmatrix}
& (n > 0)\\
\\
&
\begin{bmatrix}
1      &        & \\
2      & 3      & \huge{0} \\
\vdots & \vdots & \ddots \\
m-|n|+1  & m-|n|+2  & \cdots &  m \\
\end{bmatrix}
& (n < 0)\\
\end{cases}
,\end{align*}
$$

[ja]: #
ただし $m=|n|(|n|+1)/2$ である。

[en]: #
where $m=|n|(|n|+1)/2$.

[END]: #
- $n$: `int`
- $1 \leq |n| \leq 10^{3}$
- Return: `np.ndarray`
  - `shape`: `(n, n)`
  - `dtype`: `np.int32`
- Sample
  - `2`->`array([[1, 2], [0, 3]], dtype=int32)`
  - `-3`->`array([[1, 0, 0], [2, 3, 0], [4, 5, 6]], dtype=int32)`

[tips]: #
- [`np.triu`](https://numpy.org/doc/stable/reference/generated/numpy.triu.html)
- [`np.tril`](https://numpy.org/doc/stable/reference/generated/numpy.tril.html)
- [`np.triu_indices_from`](https://numpy.org/doc/stable/reference/generated/numpy.triu_indices_from.html)
- [`np.tril_indices_from`](https://numpy.org/doc/stable/reference/generated/numpy.tril_indices_from.html)

[/tips]: #

In [ ]:
def solution(n):
    # BEGIN
    n_abs = abs(n)
    m = n_abs * (n_abs + 1) / 2
    mat = np.zeros((n_abs, n_abs), dtype=np.int32)
    arr = np.arange(1, m + 1, dtype=np.int32)
    if n > 0:
        mat[np.triu_indices_from(mat)] = arr
    elif n < 0:
        mat[np.tril_indices_from(mat)] = arr
    return mat
    # END


test_func(solution, "01_10")
# show_solution("01_10")  # Uncomment it to see the solution.

[ja]: #
## 2. 配列の変形

[en]: #
## 2. Array manipulation

[ja]: #
次に配列の変形方法を扱います。
NumPyでは例えば`np.reshape`を用いて以下のように配列の変形できます。
```py
arr = np.arange(24)
arr = arr.reshape(4, 6)
```
`arr`が1次元配列から2次元配列、すなわち行列に変換されています。
この節ではこうした配列の変形方法を網羅的に紹介します。

[en]: #
Next we examine array reshaping and dimension manipulation.
Use `np.reshape` to change the shape of an array, for example:

```py
arr = np.arange(24)
arr = arr.reshape(4, 6)
```

This converts a 1-D array into a 2-D matrix.
The section covers related techniques for inserting, removing, and permuting axes.

Q2.1.

[ja]: #
$n$ 枚の高さ $h$ ・幅 $w$ ピクセルのRGBA画像の情報が格納されたテンソル $A$ が与えられる。
すなわち $a_{ijk}$ は $i$ 番目の画像の $(j, k)$ 座標が持つRGBA値で、`Red`, `Green`, `Blue`, `Alpha`の順で4次元ベクトルとして保存されている。
今アルファ値を抜いたRGBのみを使用し、各画像の情報を $p(=h \times w \times 3)$ 次元のベクトルとして表現したい。
すなわちテンソル $A$ を行列 (2次元テンソル) $B\in \mathbb{R}^{n\times p}$に変換するコードを書け。

[en]: #
A tensor $A$ containing information for $n$ RGBA images, each of height $h$ and width $w$ pixels, is given.
Specifically, $a_{ijk}$ represents the RGBA value at the $(j, k)$ coordinate of the $i$-th image, stored as a 4-dimensional vector in the order of `Red`, `Green`, `Blue`, `Alpha`.
Now, we want to keep only the RGB values, omitting the Alpha channel, and represent each image's information as a vector of length $p(=h \times w \times 3)$.
Write code to transform the tensor $A$ into a matrix (2-dimensional tensor) $B\in \mathbb{R}^{n\times p}$.

[END]: #
- $A$: `np.ndarray`
  - `shape`: `(n, h, w, 4)`
  - `dtype`: `np.uint8`
- $1 \leq n \leq 10^{2}$
- $1 \leq w, h \leq 10^{2}$
- $0 \leq a_{ijkl} \leq 255$
- Return: `np.ndarray`
  - `shape`: `(n, h * w * 3)`
  - `dtype`: `np.uint8`

[tips]: #
- [`np.shape`](https://numpy.org/doc/stable/reference/generated/numpy.shape.html)
- [`np.reshape`](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html)

[/tips]: #

In [ ]:
def solution(arr):
    # BEGIN
    return arr[..., :-1].reshape(arr.shape[0], -1)

    # Alternative solution.
    # n = arr.shape[0]
    # p = arr.shape[1] * arr.shape[2] * 3
    # return arr[:, :, :, :-1].reshape(n, p)
    # END


test_func(solution, "02_01")
# show_solution("02_01")  # Uncomment it to see the solution.

Q2.2.

[ja]: #
$t$ ステップにわたり座標XYZに対応する $w \times h \times d$ グリッドの各点に対応するセンサ値 (16bit整数)を格納した4次元テンソル $A\in \mathbb{R}^{t\times w \times h \times d}$ を考える。
今座標系ZXYでの問題に応用するため $B_{ijkl}=A_{iljk}$ となるような4次元テンソル $B$ を構築したい。
そのような変換を実現するコードを書け。

[en]: #
Consider a 4-dimensional tensor $A\in \mathbb{R}^{t\times w \times h \times d}$ that stores sensor values (16-bit integers) of XYZ coordinates for each point on a $w \times h \times d$ grid, over $t$ time steps.
To use this data in the ZXY coordinate system, we want to construct a 4-dimensional tensor $B$ such that $B_{ijkl}=A_{iljk}$.
Write code to perform this transformation.

[END]: #
- $A$: `np.ndarray`
  - `shape`: `(t, w, h, d)`
  - `dtype`: `np.uint16`
- $1 \leq t, w, h, d \leq 10^{2}$
- Return: `np.ndarray`
  - `shape`: `(t, d, w, h)`
  - `dtype`: `np.uint16`

[tips]: #
- [`np.moveaxis`](https://numpy.org/doc/stable/reference/generated/numpy.moveaxis.html)
- [`np.swapaxes`](https://numpy.org/doc/stable/reference/generated/numpy.swapaxes.html)
- [`np.transpose`](https://numpy.org/doc/stable/reference/generated/numpy.transpose.html)

[/tips]: #

In [ ]:
def solution(arr):
    # BEGIN
    return np.moveaxis(arr, -1, 1)
    # END


test_func(solution, "02_02")
# show_solution("02_02")  # Uncomment it to see the solution.

Q2.3.

[ja]: #
正の整数 $k(\geq 2)$ と、時間 $t$ に渡って $p$ 次元ベクトルを記録した行列 $A\in\mathbb{R}^{t \times p}$ が与えられる。
$A$は2次元テンソルであるが、今これを $b_{i,\underbrace{0,~\ldots,~0}_{k-2},j}=a_{ij}$ となる $k$ 次元テンソル $B\in\mathbb{R}^{t\times \underbrace{1 \times \cdots \times 1}_{k-2}\times p}$ に変換したい。
このような変換を実現するコードを書け。

[en]: #
A positive integer $k(\geq 2)$ and a matrix $A\in\mathbb{R}^{t \times p}$ that records $p$-dimensional vectors over $t$ time steps are given.
Here, $A$ is a 2-dimensional tensor.
We want to transform $A$ into a $k$-dimensional tensor $B\in\mathbb{R}^{t\times \underbrace{1 \times \cdots \times 1}_{k-2}\times p}$ such that $b_{i,\underbrace{0,~\ldots,~0}_{k-2},j}=a_{ij}$.
Write code to perform this transformation.

[END]: #
- $A$: `np.ndarray`
  - `shape`: `(t, p)`
  - `dtype`: `np.int64`
- $2 \leq k \leq 10$
- $1 \leq t, p \leq 10^{2}$
- Return: `np.ndarray`
  - `shape`: `(t, ... ,p)`
  - `dtype`: `np.int64`
- Sample
  - `3, array([[1, 2], [3, 4]])`->`array([[[1, 2]], [[3, 4]]])`

[tips]: #
- [`np.expand_dims`](https://numpy.org/doc/stable/reference/generated/numpy.expand_dims.html)

[/tips]: #

In [ ]:
def solution(k, arr):
    # BEGIN
    for _ in range(k - 2):
        arr = np.expand_dims(arr, axis=1)
    return arr

    # Alternative solution 1.
    # for _ in range(k - 2):
    #     arr = arr[:, None, ..., :]
    # return arr

    # Alternative solution 2.
    # sli = (slice(None),) + (None,) * (k - 2) + (slice(None),)
    # return arr[sli]

    # Alternative solution 3.
    # t, p = arr.shape
    # return arr.reshape(t, *(1,) * (k - 2), p)
    # END


test_func(solution, "02_03")
# show_solution("02_03")  # Uncomment it to see the solution.

Q2.4.

[ja]: #
正の整数 $k, m, n$ が与えられる。
$1$ から $k^2$ を並べた以下の正方行列 $S\in\mathbb{R}^{k\times k}$ を考える。

[en]: #
Some positive integers $k, m, n$ are given.
Consider a square matrix $S\in\mathbb{R}^{k\times k}$ arranged with numbers from $1$ to $k^2$ as follows:

[END]: #
$$
\begin{align*}
\begin{bmatrix}
1 & 2 & \cdots & k-1 & k \\
k+1 & k+2 & \cdots & 2k-1 & 2k \\
\vdots & \vdots & \ddots & \vdots & \vdots \\
k^2-k+1  & k^2-k+2 & \cdots & k^2-1 & k^2 \\
\end{bmatrix}
.\end{align*}
$$

[ja]: #
このSを列方向に $m$ 個、行方向に $n$ 個並べたブロック行列 $T \in \mathbb{R}^{km\times kn}$ を出力するコードを書け。

[en]: #
Write code to output the block matrix $T \in \mathbb{R}^{km\times kn}$ formed by arranging $S$ $m$ times in the column direction and $n$ times in the row direction.

[END]: #
- $k, m, n$: `int`
- $1 \leq k, n, m \leq 10$
- Return: `np.ndarray`
  - `shape`: `(k * m, k * n)`
  - `dtype`: `np.int64`
- Sample
```py
In [*]: print(str(solution(2, 3, 4)))
[[1 2 1 2 1 2 1 2]
 [3 4 3 4 3 4 3 4]
 [1 2 1 2 1 2 1 2]
 [3 4 3 4 3 4 3 4]
 [1 2 1 2 1 2 1 2]
 [3 4 3 4 3 4 3 4]]
```

[tips]: #
- [`np.tile`](https://numpy.org/doc/stable/reference/generated/numpy.tile.html)
- [`np.repeat`](https://numpy.org/doc/stable/reference/generated/numpy.repeat.html)

[/tips]: #

In [ ]:
def solution(k, m, n):
    # BEGIN
    return ((np.arange(m * k) % k) * k)[:, None] + (np.arange(n * k) % k)[None, :] + 1

    # Alternative solution.
    # s = np.arange(1, k * k + 1).reshape(k, k)
    # return np.tile(s, (m, n))
    # END


test_func(solution, "02_04")
# show_solution("02_04")  # Uncomment it to see the solution.

Q2.5.

[ja]: #
時間 $k \in [0, t]$ にわたって $n$ 個のドローンの位置のXYZ座標を記した3次元テンソル $A\in\mathbb{R}^{(t+1)\times n \times 3}$ が与えられる。
今時刻 $k+1$ と $k$ の座標の差から時刻 $k+1$ における速度ベクトルと、それをまとめた3次元テンソル $B\in\mathbb{R}^{t\times n \times 3}$ が得られる。
時間 $k\in[1, t]$ の位置と速度をまとめて6次元ベクトルとしてまとめた3次元テンソル $C\in\mathbb{R}^{t\times n \times 6}$ を構成するコードを書け。

[en]: #
A 3-dimensional tensor $A\in\mathbb{R}^{(t+1)\times n \times 3}$ is given, which records the XYZ coordinates of $n$ drones over time $k \in [0, t]$.
Now, we can obtain the velocity vector by computing the difference between the coordinates at time $k+1$ and $k$, and this data is arranged in a 3-dimensional tensor $B\in\mathbb{R}^{t\times n \times 3}$.
Write code to construct a 3-dimensional tensor $C\in\mathbb{R}^{t\times n \times 6}$, which combines the position and velocity into a 6-dimensional vector for each time $k\in[1, t]$.

[END]: #
- $A$: `np.ndarray`
  - `shape`: `(t + 1, n, 3)`
  - `dtype`: `np.int64`
- $1 \leq t \leq 10^{3}$
- $1 \leq n \leq 10$
- Return: `np.ndarray`
  - `shape`: `(t, n, 6)`
  - `dtype`: `np.int64`

[tips]: #
- [`np.concatenate`](https://numpy.org/doc/stable/reference/generated/numpy.concatenate.html)

[/tips]: #

In [ ]:
def solution(arr):
    # BEGIN
    vel = arr[1:] - arr[:-1]
    return np.concatenate([arr[1:], vel], axis=-1)
    # END


test_func(solution, "02_05")
# show_solution("02_05")  # Uncomment it to see the solution.

[ja]: #
## 3. 論理演算と検索

[en]: #
## 3. Logical operations and search

[ja]: #
次に論理演算と検索方法を扱います。
NumPyでは論理演算とインデクシングを用い以下のようなコードが可能です。
```py
arr = np.arange(10)
arr[arr >= 5] = 5
```
これは5以上の要素に5を代入するコードです。
このように配列の検索と代入、操作を直観的に記述できます。
また`np.argsort`等の関数を通して、値の代表的な検索方法を学びましょう。

[en]: #
Next, we will cover logical operations and search methods.
In NumPy, you can execute the following code involving logical operations and indexing:

```py
arr = np.arange(10)
arr[arr >= 5] = 5
```

Executing this code assigns the value 5 to all elements greater than or equal to 5.
In this way, NumPy allows you to intuitively write code for searching, assigning, and manipulating array elements.
Furthermore, let's learn typical methods for searching values using functions such as `np.argsort`.

Q3.1.

[ja]: #
正の整数 $h,w,x,y,r$ が与えられる。
今高さ$h$ px、幅$w$ pxのディスプレイ上に円を描画する処理を行列 $A=(a_{ij})_{0\leq i < h, 0\leq j < w}$ を用いて表現する。
$a_{ij}$がXY平面上の座標 $(i, j)$ に対応しているとし、以下の条件を満たす行列 $A\in\mathbb{R}^{h \times w}$ を作成するコードを書け。

[en]: #
Some positive integers $h, w, x, y, r$ are given.
Now, the task of drawing a circle on a display with height $h$ px and width $w$ px is represented by the matrix $A=(a_{ij})_{0\leq i < h, 0\leq j < w}$.
Each $a_{ij}$ corresponds to the coordinate $(i, j)$ on the XY plane.
Write code to create a matrix $A\in\mathbb{R}^{h \times w}$ that satisfies the following condition:

[END]: #
$$
\begin{align*}
a_{ij}=\begin{cases}
255 & ((i - x)^2 + (j - y)^2 \leq r^2) \\
0 & (\text{otherwise})
\end{cases}
.\end{align*}
$$

- $h, w, x, y, r$: `int`
- $10^2 \leq w, h \leq 10^3$
- $0 \leq x < h,~0 \leq y < w$
- $0 \leq r < \text{min}(h, w)$
- Return: `np.ndarray`
  - `shape`: `(h, w)`
  - `dtype`: `np.uint8`

[tips]: #
- [`np.meshgrid`](https://numpy.org/doc/stable/reference/generated/numpy.meshgrid.html)

[/tips]: #

In [ ]:
def solution(h, w, x, y, r):
    # BEGIN
    arr = np.zeros((h, w), dtype=np.uint8)
    xs, ys = np.meshgrid(np.arange(h), np.arange(w), indexing="ij")
    dist = (xs - x) ** 2 + (ys - y) ** 2

    # Alternative implementation for `dist`.
    # dist = (np.arange(h)[:, None] - x)**2 + (np.arange(w)[None, :] - y)**2

    circle = dist <= r**2
    arr[circle] = 255
    return arr
    # END


# Use the following codes to debug your implementation.
# import matplotlib.pyplot as plt
# plt.imshow(solution(200, 200, 40, 50, 30), cmap='gray', vmax=255)

test_func(solution, "03_01")
# show_solution("03_01")  # Uncomment it to see the solution.

Q3.2.

[ja]: #
正の整数 $h,w,x,y,r,s,t,u,v$ が与えられる。
今前問の設定に引き続き長方形 $L=\{(i, j)~|~s \leq i \leq t \land u \leq j \leq v\}$ となる領域を描画したい。
ただし円 $C=\{(i, j)~|~(i-x)^2+(j-y)^2 \leq r^2\}$ と重複する領域 $D=C\cap L$ は別の色で表現したい。
これらの仕様を満たすため、以下の条件を満たす行列 $A\in\mathbb{R}^{h \times w}$ を作成するコードを書け。

[en]: #
Some positive integers $h, w, x, y, r, s, t, u, v$ are given.
Continuing on from the previous question, we want to draw in addition a rectangle $L=\{(i, j)~|~s \leq i \leq t \land u \leq j \leq v\}$.
However, the overlapping region of the circle $C=\{(i, j)~|~(i-x)^2+(j-y)^2 \leq r^2\}$ and the rectangle, $D=C\cap L$, should be represented in a different color.
To fulfill these specifications, write code to create a matrix $A\in\mathbb{R}^{h \times w}$ that satisfies the following conditions:

[END]: #
$$
\begin{align*}
a_{ij}=\begin{cases}
255 & ((i, j) \in D ) \\
127 & ((i, j) \in C \setminus D ) \\
127 & ((i, j) \in L \setminus D ) \\
0 & (\text{otherwise})
\end{cases}
.\end{align*}
$$

- $h, w, x, y, r, s, t, u, v$: `int`
- $10^2 \leq w, h \leq 10^3$
- $0 \leq x < h,~0 \leq y < w$
- $0 \leq r < \text{min}(h, w)$
- $0 \leq s < t < h,~0 \leq u < v < w$
- Return: `np.ndarray`
  - `shape`: `(h, w)`
  - `dtype`: `np.uint8`

[tips]: #
- [`Logical operations`](https://numpy.org/doc/stable/reference/routines.logic.html#logical-operations)

[/tips]: #

In [ ]:
def solution(h, w, x, y, r, s, t, u, v):
    # BEGIN
    arr = np.zeros((h, w), dtype=np.uint8)
    xs, ys = np.meshgrid(np.arange(h), np.arange(w), indexing="ij")
    dist = (xs - x) ** 2 + (ys - y) ** 2
    circle = dist <= r**2

    rect_x = (s <= xs) & (xs <= t)
    rect_y = (u <= ys) & (ys <= v)
    rect = rect_x & rect_y

    arr[rect & circle] = 255
    arr[rect ^ circle] = 127
    return arr
    # END


# Use the following codes to debug your implementation.
# import matplotlib.pyplot as plt
# plt.imshow(solution(200, 200, 60, 50, 30, 10, 100, 30, 50), cmap='gray', vmax=255)

test_func(solution, "03_02")
# show_solution("03_02")  # Uncomment it to see the solution.

Q3.3.

[ja]: #
前節で登場した整数をone-hotベクトル化する操作の逆変換を実装しよう。
行列 $A\in\mathbb{R}^{n\times 10}$ が与えられる。
$A$ の $i$ 行目 $a_i=[a_{i0},a_{i1},~\ldots,~a_{i9}]$ はいずれかの要素が`1`でそれ以外が`0`の単位ベクトルである。
各 $i$ に対して $a_{ik}=1$ となる $k$ を並べた配列を出力するコードを書け。

[en]: #
In the previous section, we introduced the one-hot vectorization operation.
Let's implement the inverse transformation of this operation.
A matrix $A\in\mathbb{R}^{n\times 10}$ is given.
The $i$-th row $a_i=[a_{i0},a_{i1},~\ldots,~a_{i9}]$ of $A$ is a unit vector where one element is `1` and the rest are `0`.
Write code to output an array containing $k$ for each $i$, where $a_{ik}=1$.

[END]: #
- $A$: `np.ndarray`
  - `shape`: `(n, 10)`
  - `dtype`: `np.uint8`
- $1 \leq n \leq 10^{3}$
- Return: `np.ndarray`
  - `shape`: `(n,)`
  - `dtype`: `np.uint8`
- Sample
  - `array([[0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]])`->`array([2, 3])`

[tips]: #
- [`np.argmax`](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html)
- [`np.argmin`](https://numpy.org/doc/stable/reference/generated/numpy.argmin.html)

[/tips]: #

In [ ]:
def solution(arr):
    # BEGIN
    return np.argmax(arr, axis=1)
    # END


test_func(solution, "03_03")
# show_solution("03_03")  # Uncomment it to see the solution.

Q3.4.

[ja]: #
行列 $A=(a_{ij})_{0\leq i < m, 0 \leq j < n}$ と正の整数 $k$ が与えられる。
$a_{ij}$ の値を降順に並べた時、上位 $k$ 個のインデックス $(i, j)$ を順に取得したい。
そのような処理を実現するコードを書け。

[en]: #
A matrix $A=(a_{ij})_{0\leq i < m, 0 \leq j < n}$ and a positive integer $k$ is given.
Write code to retrieve the indices $(i, j)$ of the top $k$ highest values in $A$, sorted in descending order.

[END]: #
- $A$: `np.ndarray`
  - `shape`: `(m, n)`
  - `dtype`: `np.float64`
- $k$: `int`
- $1 \leq m, n \leq 10^{2}$
- $1 \leq k \leq \text{min}(mn, 10^2)$
- Return: `np.ndarray`
  - `shape`: `(k, 2)`
  - `dtype`: `np.int64`
- Sample
  - `array([[3., 7., 4.], [6., 5., 8]])`->`array([[1, 2],[0, 1]])`

[tips]: #
- [`np.argsort`](https://numpy.org/doc/stable/reference/generated/numpy.argsort.html)
- [`np.unravel_index`](https://numpy.org/doc/stable/reference/generated/numpy.unravel_index.html)

[/tips]: #

In [ ]:
def solution(arr, k):
    # BEGIN
    val = np.argsort(arr, axis=None)
    return np.array(np.unravel_index(val[-k::], arr.shape)).T[::-1]
    # END


test_func(solution, "03_04")
# show_solution("03_04")  # Uncomment it to see the solution.

Q3.5.

[ja]: #
行列 $A, B\in\mathbb{R}^{m\times n}$ が与えられる。
$A$ と $B$ の各要素に関して大きい方を抽出した行列 $C=(\text{max}(a_{ij}, b_{ij}))_{0\leq i < m, 0\leq j < n}$ を作成するコードを書け。

[en]: #
Matrices $A, B\in\mathbb{R}^{m\times n}$ are given.
Write code to create a matrix $C=(\text{max}(a_{ij}, b_{ij}))_{0\leq i < m, 0\leq j < n}$, which contains the larger element from $A$ and $B$ for each corresponding position.

[END]: #
- $A, B$: `np.ndarray`
  - `shape`: `(m, n)`
  - `dtype`: `np.int64`
- $1 \leq m, n \leq 10^{2}$
- Return: `np.ndarray`
  - `shape`: `(m, n)`
  - `dtype`: `np.int64`

[tips]: #
- [`np.where`](https://numpy.org/doc/stable/reference/generated/numpy.where.html)
- [`np.maximum`](https://numpy.org/doc/stable/reference/generated/numpy.maximum.html)

[/tips]: #

In [ ]:
def solution(arr_a, arr_b):
    # BEGIN
    return np.where(arr_a > arr_b, arr_a, arr_b)

    # Alternative solution.
    return np.maximum(arr_a, arr_b)
    # END


test_func(solution, "03_05")
# show_solution("03_05")  # Uncomment it to see the solution.

[ja]: #
## 4. 数理と統計

[en]: #
## 4. Math and statistics

[ja]: #
次にいくつかの主要な数学、ならびに統計関数を扱います。
NumPyでは、`np.sin, np.exp`等の初等関数のほか、最大、最小値を集計する`np.min, np.max`、平均、分散を計算する`np.mean, np.var`等の統計関数が提供されています。
これらは今後の章でも頻出の重要な関数ですので、使い方をマスターしましょう。

[en]: #
Next, we will learn some important mathematical and statistical functions.
NumPy provides elementary functions such as `np.sin` and `np.exp`, as well as statistical functions like `np.min` and `np.max` (for obtaining the minimum and maximum values) and `np.mean` and `np.var` (for calculating the mean and variance).
These functions will appear repeatedly in later chapters, so let's master how to use them.

Q4.1.

[ja]: #
$t$ ステップにわたり $n$ 台のドローンの原点周りのXYZ座標が3次元テンソル $A\in\mathbb{R}^{t\times n \times 3}$ として与えられる。
各ドローンの原点周りの極座標を計算し、同じサイズの3次元テンソル $B\in\mathbb{R}^{t\times n \times 3}$ として計算結果を出力したい。
なおXYZ座標 $(x,y,z)$ から極座標 $(r,\theta,\phi)$ への変換は[以下の式](https://en.wikipedia.org/wiki/Spherical_coordinate_system#Coordinate_system_conversions)で計算される。

[en]: #
Over $t$ time steps, the XYZ coordinates of $n$ drones around the origin are given as a 3-dimensional tensor $A\in\mathbb{R}^{t\times n \times 3}$.
We want to obtain the spherical coordinates around the origin of each drone, and output the results in a 3-dimensional tensor $B\in\mathbb{R}^{t\times n \times 3}$ of the same size.
The transformation from XYZ coordinates $(x,y,z)$ to spherical coordinates $(r,\theta,\phi)$ is calculated using [the following equations](https://en.wikipedia.org/wiki/Spherical_coordinate_system#Coordinate_system_conversions):

[END]: #
$$
\begin{align*}
r &= \sqrt{x^2 + y^2 + z^2} \\
\theta &= \mathrm{Arccos}{\frac{z}{\sqrt{x^2 + y^2 + z^2}}} \\
\phi &= \mathrm{sgn}(y)\mathrm{Arccos}{\frac{x}{\sqrt{x^2 + y^2}}}
.\end{align*}
$$

[ja]: #
このような処理を実現するコードを書け。

[en]: #
Write code to perform this computation.

[END]: #
- $A$: `np.ndarray`
  - `shape`: `(t, n, 3)`
  - `dtype`: `np.float64`
- $10^2 \leq t \leq 10^3$
- $1 \leq n \leq 10^2$
- $0.1 \leq \sqrt{\sum_{k=0}^{1} a_{ijk}^2} \leq \sqrt{\sum_{k=0}^{2} a_{ijk}^2} \leq 10$
- Return: `np.ndarray`
  - `shape`: `(t, n, 3)`
  - `dtype`: `np.float64`

[tips]: #
- [Mathematical functions](https://numpy.org/doc/stable/reference/routines.math.html)
- [`np.arccos`](https://numpy.org/doc/stable/reference/generated/numpy.arccos.html)
- [`np.sign`](https://numpy.org/doc/stable/reference/generated/numpy.sign.html)
- [`np.cumsum`](https://numpy.org/doc/stable/reference/generated/numpy.cumsum.html)

[/tips]: #

In [ ]:
def solution(arr):
    # BEGIN
    out = np.zeros_like(arr)
    r_xy = np.linalg.norm(arr[..., :2], axis=-1)
    r_xyz = np.linalg.norm(arr, axis=-1)
    th = np.arccos(arr[..., 2] / r_xyz)
    ph = np.sign(arr[..., 1]) * np.arccos(arr[..., 0] / r_xy)
    out[..., 0] = r_xyz
    out[..., 1] = th
    out[..., 2] = ph
    return out
    # END


test_func(solution, "04_01")
# show_solution("04_01")  # Uncomment it to see the solution.

Q4.2.

[ja]: #
オリンピックの採点競技ではしばしば、最大と最小を除いた得点を反映するトリム平均法が採用されている。
いま $n$ 人の選手に対する10人の審査員の点数のデータが行列 $A\in\mathbb{R}^{n\times 10}$ として与えられる。
各選手に対して最大値と最小値を除いた8つの点数の合計をその選手の最終評価とする。
与えられたデータに対して $n$ 人の選手の最終評価を格納した配列を出力するコードを書け。

[en]: #
In the Olympic Games, the trimmed mean method, which excludes the highest and lowest scores, is often used for scoring.
Now, data for $n$ athletes scored by 10 judges is provided as a matrix $A\in\mathbb{R}^{n\times 10}$.
For each athlete, the final score is determined by the sum of the 8 scores excluding the highest and lowest scores.
Write code to output an array containing the final scores for the $n$ athletes.

[END]: #
- $A$: `np.ndarray`
  - `shape`: `(n, 10)`
  - `dtype`: `np.int64`
- $1 \leq n \leq 10^{3}$
- $0 \leq a_{ij} \leq 10^2$
- Return: `np.ndarray`
  - `shape`: `(n,)`
  - `dtype`: `np.int64`
- Sample
  - `array([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]])`->`array([44])`
  - `array([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])`->`array([5])`

[tips]: #
- [`np.min`](https://numpy.org/doc/stable/reference/generated/numpy.min.html)
- [`np.max`](https://numpy.org/doc/stable/reference/generated/numpy.max.html)

[/tips]: #

In [ ]:
def solution(arr):
    # BEGIN
    return arr.sum(axis=-1) - arr.min(axis=-1) - arr.max(axis=-1)
    # END


test_func(solution, "04_02")
# show_solution("04_02")  # Uncomment it to see the solution.

Q4.3.

[ja]: #
正の整数 $m$ と $t$ ステップのセンサの値を記録した時系列データが格納された配列 $A\in\mathbb{R}^{t}$ 与えられる。
いまノイズに対する影響を最小限にするため、以下のような平滑化処理が施されて得られた配列 $B=(b_{i})$ を作成したい。

[en]: #
A positive integer $m$ and an array $A\in\mathbb{R}^{t}$ containing sensor data over $t$ time steps are given.
To minimize the impact of noise, we want to create a new array $B=(b_{i})$ by applying the smoothing process as shown below:

[END]: #
$$
\begin{align*}
b_{i} &= \frac{1}{m}\sum\limits_{u=0}^{m-1} a_{i-u}\\
a_{i} &= \begin{cases}
  a_{i} & (i \geq 0) \\
  a_{0} & (\text{otherwise})
\end{cases}
.\end{align*}
$$

[ja]: #
このような処理を実現するコードを書け。

[en]: #
Write code to output the smoothed data array.

[END]: #
- $m$: `int`
- $A$: `np.ndarray`
  - `shape`: `(t,)`
  - `dtype`: `np.float64`
- $1 \leq m \leq t \leq 10^{3}$
- $0 \leq |a_{ij}| \leq 10$
- Return: `np.ndarray`
  - `shape`: `(t,)`
  - `dtype`: `np.float64`
- Sample
  - `3, np.array([1., 1., 1., 4., 7.])`->`array([1., 1., 1., 2., 4.])`

[tips]: #
- [`np.convolve`](https://numpy.org/doc/stable/reference/generated/numpy.convolve.html)

[/tips]: #

In [ ]:
def solution(m, arr):
    # BEGIN
    out = np.pad(arr, (m - 1, 0), mode="edge")
    return np.convolve(out, np.full(m, 1 / m), mode="valid")
    # END


test_func(solution, "04_03")
# show_solution("04_03")  # Uncomment it to see the solution.

Q4.4.

[ja]: #
$t$ ステップにわたり $p$ 種類のセンサの予測データと正解データセットを記した行列 $Y=(y_{ij}), D=(d_ij) \in \mathbb{R}^{t\times p}$ が与えられる。
今各センサに関して、NRMSE (Normalized Root Mean Squared Error)を求めたい。ただし $k~(0\leq k < p)$ 番目のデータに対するNRMSEは以下の式の定義を採用する。

[en]: #
Matrices $Y=(y_{ij}), D=(d_{ij}) \in \mathbb{R}^{t\times p}$, containing predicted data and true data for $p$ types of sensors over $t$ time steps, are given.
We want to calculate the normalized root mean squared error (NRMSE) for each sensor.
The NRMSE for the $k$-th data (where $0\leq k < p$) is defined by the following equation:

[END]: #
$$
\begin{align*}
\mathrm{NRMSE}_k &:= \frac{\mathrm{RMSE}_k}{\sigma_k}\\
\mathrm{RMSE}_k &:= \sqrt{\mathrm{MSE}_k} \\
&=\sqrt{\frac{1}{t}\sum\limits_{u=0}^{t-1}(y_{uk}-d_{uk})^2}\\
\sigma_k &:= \sqrt{\mathrm{Var}[d_k]} \\
&=\sqrt{\frac{1}{t}\sum\limits_{u=0}^{t-1} (d_{uk} - \mu_k)^2} \\
\mu_k &:=\frac{1}{t}\sum\limits_{u=0}^{t-1} d_{uk}
.\end{align*}
$$

[ja]: #
NRMSEをまとめた要素数 $p$ の配列を出力するコードを書け。

[en]: #
Write code to output an array of size $p$, containing the NRMSE for each sensor.

[END]: #
- $Y, D$: `np.ndarray`
  - `shape`: `(t, p)`
  - `dtype`: `np.float64`
- $10^2 \leq t \leq 10^{3}$
- $1 \leq p \leq 10^{2}$
- $0 \leq |d_{ij}|, |y_{ij}| \leq 10$
- Return: `np.ndarray`
  - `shape`: `(p,)`
  - `dtype`: `np.float64`

[tips]: #
- [`np.mean`](https://numpy.org/doc/stable/reference/generated/numpy.mean.html)
- [`np.average`](https://numpy.org/doc/stable/reference/generated/numpy.average.html)
- [`np.var`](https://numpy.org/doc/stable/reference/generated/numpy.var.html)
- [`np.std`](https://numpy.org/doc/stable/reference/generated/numpy.std.html)
- [`np.sqrt`](https://numpy.org/doc/stable/reference/generated/numpy.sqrt.html)

[/tips]: #

In [ ]:
def solution(y, d):
    # BEGIN
    mse = ((y - d) ** 2).mean(axis=0)
    std = np.std(d, axis=0)
    return np.sqrt(mse) / std

    # Alternative solution.
    # return (((y - d) ** 2).sum(axis=0) / ((d - d.mean(axis=0)) ** 2).sum(axis=0)) ** 0.5
    # END


test_func(solution, "04_04")
# show_solution("04_04")  # Uncomment it to see the solution.

Q4.5.

[ja]: #
$n$ 個の1桁の整数 (0~9)からなる配列 $A$ が与えられる。
今この配列から数字 $k$ の登場頻度を計算し確率分布 $P(k)$ を構成する。
そして以下の式によって定義されるエントロピー $H_2(P)$ を計算したい。

[en]: #
An array $A$ consisting of $n$ single-digit integers (0 to 9) is given.
Our aim is to count the frequency of each digit $k$ and obtain the probability distribution $P(k)$.
Then, we will calculate the entropy $H_2(P)$ defined by the following formula:

[END]: #
$$
\begin{align*}
H_2(P):=-\sum\limits_{k=0}^9 P(k)\log_2(P(k))
.\end{align*}
$$

[ja]: #
このような処理を実現するコードを書け。
ただし0除算の発生に留意せよ。

[en]: #
Write code to perform this computation.
Be careful to avoid division by zero.

[END]: #
- $A$: `np.ndarray`
  - `shape`: `(n,)`
  - `dtype`: `np.int8`
- $1 \leq n \leq 10^{4}$
- Return: `float`
- Sample
  - `array([0, 1, 0, 1])`->`1.0`
  - `array([0, 1, 2, 3])`->`2.0`
  - `array([0, 0])`->`0.0`

[tips]: #
- [`np.bincount`](https://numpy.org/doc/stable/reference/generated/numpy.bincount.html)
- [`np.log`](https://numpy.org/doc/stable/reference/generated/numpy.log.html)
- [`np.log2`](https://numpy.org/doc/stable/reference/generated/numpy.log2.html)
- [`np.nansum`](https://numpy.org/doc/stable/reference/generated/numpy.nansum.html)

[/tips]: #

In [ ]:
def solution(arr):
    # BEGIN
    hist = np.bincount(arr)
    dist = hist[hist > 0] / hist.sum()
    out = dist * np.log2(dist)
    return float(-np.nansum(out))
    # END


test_func(solution, "04_05")
# show_solution("04_05")  # Uncomment it to see the solution.

[ja]: #
## 5. 線形代数

[en]: #
## 5. Linear algebra

[ja]: #
最後にNumPyのもっとも強力な部分にして、この教材でも重要である線形代数に関連するNumPyのライブラリを扱いましょう。
本章では特に最小二乗法を中心に、逆行列や特異値分解、固有値の計算等、重要な行列演算の使用法を学びます。

[en]: #
Finally, let's learn how to use NumPy methods related to linear algebra.
These methods are among the most powerful components of NumPy and play a vital role in this material as well.
In this chapter, we will focus particularly on the least squares method, while also covering major matrix operations such as computing inverse matrices, performing singular value decomposition, and finding eigenvalues.

Q5.1.

[ja]: #
$t$ ステップにわたる $n$ 台のドローンの原点周りのXYZ座標が記録された3次元テンソル $A\in\mathbb{R}^{t\times n \times 3}$ 、観測者の位置座標を記した行列 $O\in\mathbb{R}^{t\times 3}$ 、ならびにZ軸回りに観測者の正面方向の角度を記した配列 $\Theta \in\mathbb{R}^{t}$ が与えられる。

今観測者の位置を原点に観測者の正面方向をX'軸に据えるX'Y'Z'座標系を考える。
このX'Y'Z'座標系上での各時刻のドローンの座標を格納した3次元テンソル $B\in\mathbb{R}^{t\times n \times 3}$ を計算するコードを書け。
ただし観測者の正面方向の角度が $\theta$ の時、Z軸回りの座標変換行列は以下の式で表される。

[en]: #
A 3-dimensional tensor $A\in\mathbb{R}^{t\times n \times 3}$ contains the XYZ coordinates of $n$ drones around the origin over $t$ time steps.
A matrix $O\in\mathbb{R}^{t\times 3}$ contains the observer's position coordinates.
An array $\Theta \in\mathbb{R}^{t}$ contains the angle of the observer's forward direction around the Z-axis.

Consider a coordinate system X'Y'Z' where the observer's position is the origin and the observer's forward direction is the X' axis.
Write code to compute a 3-dimensional tensor $B\in\mathbb{R}^{t\times n \times 3}$ containing the drone coordinates at each time step in the X'Y'Z' coordinate system.
When the angle of the observer's forward direction is $\theta$, the coordinate transformation matrix around the Z-axis is given by:

[END]: #
$$
\begin{align*}
x_\text{new} = \begin{bmatrix}
\cos \theta & \sin \theta & 0 \\
-\sin\theta & \cos \theta & 0 \\
0           & 0           & 1
\end{bmatrix} x_\text{old}
.\end{align*}
$$

[END]: #
- $A$: `np.ndarray`
  - `shape`: `(t, n, 3)`
  - `dtype`: `np.float64`
- $O$: `np.ndarray`
  - `shape`: `(t, 3)`
  - `dtype`: `np.float64`
- $\Theta$: `np.ndarray`
  - `shape`: `(t,)`
  - `dtype`: `np.float64`
- $1 \leq t \leq 10^3$
- $1 \leq n \leq 10^2$
- Return: `np.ndarray`
  - `shape`: `(t, n, 3)`
  - `dtype`: `np.float64`

[tips]: #
- [`np.matmul`](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html)

[/tips]: #

In [ ]:
def solution(arr, obs, angle):
    # BEGIN
    out = arr - obs[:, None, :]  # [t, n, 3]
    cos, sin = np.cos(angle), np.sin(angle)
    rot = cos[:, None, None, None] * np.array([[1, 0, 0], [0, 1, 0], [0, 0, 0]])
    rot += sin[:, None, None, None] * np.array([[0, 1, 0], [-1, 0, 0], [0, 0, 0]])
    rot += np.array([[0, 0, 0], [0, 0, 0], [0, 0, 1]], dtype=np.float64)
    return (rot @ out[:, :, :, None])[..., 0]

    # NOTE
    # rot: [t, 1, 3, 3]
    # out[..., None]: [t, n, 3, 1]
    # => [t, n, 3, 1] => [t, n, 3]
    # END


# Use the following codes to debug your implementation.
# arr = [[[1, 0, 0]]]
# obs = [[-1, 0, -1]]
# angle = [np.pi / 2]
# arr, obs, angle = map(np.array, [arr, obs, angle])
# solution(arr, obs, angle)  # [[[0., -2., 1]]] is expected

test_func(solution, "05_01")
# show_solution("05_01")  # Uncomment it to see the solution.

Q5.2.

[ja]: #
行列 $A\in\mathbb{R}^{n \times p}$ が与えられる。
今各行ベクトル $a_k$ に関して以下のとおり、そのL2ノルムが1になるように正規化したい。

[en]: #
A matrix $A\in\mathbb{R}^{n \times p}$ is given.
For each row vector $a_k$, we want to normalize it so that its L2 norm becomes 1, as follows:

[END]: #
$$
\begin{align*}
\|a_k\|&=\sqrt{\sum\limits_{j=0}^{p-1}a_{kj}^2} \\
&= 1
.\end{align*}
$$

[ja]: #
そのような処理を実現するコードを書け。

[en]: #
Write code to perform this computation.

[END]: #
- $A$: `np.ndarray`
  - `shape`: `(n, p)`
  - `dtype`: `np.float64`
- $1 \leq n \leq 10^{3}$
- $1 \leq p \leq 10^{3}$
- Return: `np.ndarray`
  - `shape`: `(n, p)`
  - `dtype`: `np.float64`
- Relative error: `1e-9`
- Sample
  - `2 * np.eye(3)`->`array([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]])`

[tips]: #
- [`np.linalg.norm`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html)

[/tips]: #

In [ ]:
def solution(arr):
    # BEGIN
    return arr / np.linalg.norm(arr, axis=-1, keepdims=True)
    # END


test_func(solution, "05_02")
# show_solution("05_02")  # Uncomment it to see the solution.

Q5.3.

[ja]: #
最小二乗問題は科学計算において頻出する重要な問題で、行列 $X\in\mathbb{R}^{n\times p}$ と配列 $Y\in\mathbb{R}^{n}$ が与えられた時 $\mathcal{L}(\beta)=\|X\beta-Y\|^2$ を最小化する $\beta\in\mathbb{R}^{p}$ の最適化問題として定式化される。
NumPyでは最小二乗解を計算する[`np.linalg.lstsq`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.lstsq.html)が用意されており実用上はこの使用が推奨されるが、ここでは仕組みの理解とNumPyの練習のため、これを使わずに実装しよう。

さて機械学習や統計学の分野では $X$ はデザイン行列とも呼ばれ、 $p$ 次元の説明変数を有する $n$ 個のデータを並べたものと解釈される。
今 $n>p$ すなわち十分にデータ数が確保されている状況を考える。
この時 $\mathcal{L}(\beta)$ を最小化する最小二乗解 $\hat{\beta}$ は以下の式で明示的に得られる (詳細な導出は自ら調べ各自証明を追ってほしい)。

[en]: #
The least squares problem is an important problem frequently encountered in scientific computing.
It is formulated as an optimization problem to find $\beta\in\mathbb{R}^{p}$ that minimizes $\mathcal{L}(\beta)=\|X\beta-Y\|^2$, given a matrix $X\in\mathbb{R}^{n\times p}$ and an array $Y\in\mathbb{R}^{n}$.
NumPy provides the [`np.linalg.lstsq`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.lstsq.html) function to compute the least squares solution, which is recommended for practical use.
However, here we implement it without using this function to understand the underlying mechanism and practice NumPy.

In machine learning and statistics, $X$ is also called the design matrix, which contains $n$ data points with $p$ explanatory variables each.
Consider the case where $n>p$, meaning a sufficient number of data points is available.
Under this condition, the least squares solution $\hat{\beta}$ that minimizes $\mathcal{L}(\beta)$ can be explicitly obtained by the following equation (detailed derivation is left to the reader):

[END]: #
$$
\begin{align*}
\hat{\beta} = (X^\top  X)^{-1} X^\top Y
.\end{align*}
$$

[ja]: #
またこの $\hat{\beta}$ の下の残差二乗和$\mathrm{RSS}:=\mathcal{L}(\hat{\beta})$ (Residual Sum of Squares)は以下の式で表現される。

[en]: #
In addition, the residual sum of squares (RSS) under this optimal $\hat{\beta}$, denoted as $\mathrm{RSS}:=\mathcal{L}(\hat{\beta})$, is obtained by the following equation:

[END]: #
$$
\begin{align*}
\mathrm{RSS}
&= \|X\hat{\beta} - Y\|^2 \\
&= \|X(X^\top  X)^{-1}X^\top  Y - Y\|^2 \\
&= \|\left(I - X(X^\top  X)^{-1}X^\top \right)Y\|^2
.\end{align*}
$$

[ja]: #
ここまでの説明を踏まえ、与えられた $X, Y$ に対して $\mathrm{RSS}$ を計算するコードを書け。
ただし冒頭に述べたように問題の趣旨より`np.linalg.lstsq`は用いてはならない。
また答えは `np.linalg.lstsq`の返り値を用いて検証される。

[en]: #
Based on these explanations, write code to compute $\mathrm{RSS}$ for the $X, Y$ given.
As previously mentioned, do not use `np.linalg.lstsq` for this problem.
The answers will be validated against the output of `np.linalg.lstsq`.

[END]: #
- $X$: `np.ndarray`
  - `shape`: `(n, p)`
  - `dtype`: `np.float64`
- $Y$: `np.ndarray`
  - `shape`: `(n,)`
  - `dtype`: `np.float64`
- $1 \leq p  \leq 10^2$
- $p \leq n \leq 10^{3}$
- Return: `float`

[tips]: #
- [`np.linalg.lstsq`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.lstsq.html)
- [`np.linalg.inv`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.inv.html)

[/tips]: #

In [ ]:
def solution(x, y):
    # BEGIN DO NOT USE `np.linalg.lstsq`.
    beta = np.linalg.inv(x.T @ x) @ (x.T @ y)
    err = x.dot(beta) - y
    return float((err.dot(err)))
    # END


test_func(solution, "05_03")
# show_solution("05_03")  # Uncomment it to see the solution.

Q5.4.

[ja]: #
前問に引き続き最小二乗問題に対する理解を深めよう。
最小二乗解 $\hat{\beta}$ の計算では逆行列 $(X^\top  X)^{-1}$ が登場した。
よく知られているように、逆行列の存在条件として $X^\top X$ がfull-rank、すなわち $n\geq p$ の条件下では階数が $p$ でなければならない。
また固有値の絶対値が小さいときに逆行列の計算が不安定になりやすい。
このような数値計算の制約や不安定性より、最小二乗問題の計算において逆行列を用いず、[特異値分解 (Singular Value Decomposition; SVD)](https://en.wikipedia.org/wiki/Singular_value_decomposition)を用いた方法が採用される場合が多い (実際`np.linalg.lstsq`の内部でも条件によってLAPACKのAPI`dgelsd()`を使用しているがこれはSVDを用いている)。
SVDは行列 $U\in\mathbb{R}^{n\times p}$、直交行列 $ V\in\mathbb{R}^{p\times p}$ 、ならびに対角成分に*特異値*$\sigma_i$が格納されている対角行列 $\Sigma=\mathrm{diag}(\sigma_0, \sigma_1,~\ldots,~\sigma_p)\in\mathbb{R}^{p\times p}$ を用いて以下の形式で分解する手法である。

[en]: #
Building on the previous question, let us deepen our understanding of the least squares problem.
Computing the least squares solution $\hat{\beta}$ involves the inverse matrix $(X^\top  X)^{-1}$.
As is well known, the inverse matrix exists only when $X^\top  X$ is full-rank, meaning it has rank $p$ under the condition $n \geq p$.
Moreover, the inverse matrix computation becomes unstable when the absolute values of eigenvalues are small.
Due to these numerical constraints and instabilities, methods based on [singular value decomposition (SVD)](https://en.wikipedia.org/wiki/Singular_value_decomposition) are often used instead of direct matrix inversion (indeed, `np.linalg.lstsq` internally uses LAPACK's `dgelsd()`, which employs SVD).
SVD decomposes the matrix $X$ into orthogonal matrices $U\in\mathbb{R}^{n\times p}, V\in\mathbb{R}^{p\times p}$ and a diagonal matrix $\Sigma=\mathrm{diag}(\sigma_0, \sigma_1,~\ldots,~\sigma_p)\in\mathbb{R}^{p\times p}$ containing *singular values* on its diagonal, as follows:

[END]: #
$$
\begin{align*}
X = U \Sigma V^\top
.\end{align*}
$$

[ja]: #
$\hat{\beta}$ の$\hat{\beta}$ の式中に登場する $Y$ を除いた成分は $(X^\top X)^{-1}X^\top $ は[疑似逆行列](https://en.wikipedia.org/wiki/Generalized_inverse) (以下 $X^+$ と表記)と呼ばれるが、疑似逆行列 $X^+$ はSVDを用いると以下の式のとおり表現できる (証明は同様に割愛する、各自導出を推奨する)。

[en]: #
The term $(X^\top X)^{-1}X^\top $ appearing in the expression for $\hat{\beta}$ is called the [pseudoinverse](https://en.wikipedia.org/wiki/Generalized_inverse) (denoted hereafter as $X^+$).
Using SVD, the pseudoinverse $X^+$ can be expressed as follows (the proof is omitted; readers are encouraged to derive it themselves):

[END]: #
$$
\begin{align*}
X^+=V\Sigma^+ U^\top
,\end{align*}
$$

[ja]: #
ただし $\Sigma^+$ は0以外の対角成分 (特異値)の逆数をとり再度構成された対角行列である。
さて解の定量的な評価のため、決定係数 $\mathrm{R}^2$ が使用される場合が多い。
$\mathrm{R}^2$は説明変数 $X$ がどれほど目的変数 $Y$ を構成できるかを示す値である。
$\mathrm{R}^2$ は以下の式より得られ1に近いほど構成の精度が高い。

[en]: #
where $\Sigma^+$ is a diagonal matrix constructed by taking the reciprocals of the non-zero diagonal elements (singular values).
For quantitative evaluation of the solution, the coefficient of determination $\mathrm{R}^2$ is often used.
$\mathrm{R}^2$ indicates how well the explanatory variables $X$ can explain the target variable $Y$.
$\mathrm{R}^2$ is obtained from the following formula, and values closer to 1 indicate better fit:

[END]: #
$$
\begin{align*}
\mathrm{R}^2&:=1-\frac{\mathrm{RSS}}{n\mathrm{Var}[Y]}\\
&=1-\frac{\mathrm{RSS}}{\sum\limits_{i}^{n} (y_i-\mathrm{E}[Y])^2}
.\end{align*}
$$

[ja]: #
ここでは逆行列を用いない $\mathrm{R}^2$ の計算を目指そう。
いま簡単のため $Y$ の平均値 $\mathrm{E}[Y]=\frac{1}{n}\sum\limits_{j=1}^{n}y_j$ は0となるように正規化されたものを採用する。
 $\|Y\|^2=n\mathrm{Var}[Y]$ と表記し、前問での $\mathrm{RSS}$ の式を代入すると以下の式のとして変形できる。

[en]: #
Here we aim to compute $\mathrm{R}^2$ without using the inverse matrix.
For simplicity, assume that $Y$ is normalized so that its mean $\mathrm{E}[Y]=\frac{1}{n}\sum\limits_{j=1}^{n}y_j$ equals zero.
Using the notation $\|Y\|^2=n\mathrm{Var}[Y]$ and substituting the formula for $\mathrm{RSS}$ from the previous question, we obtain:

[END]: #
$$
\begin{align*}
\mathrm{R}^2&:=1-\frac{\mathrm{RSS}}{\|Y\|^2}\\
&= 1-\frac{\|\left(I - X(X^\top  X)^{-1}X^\top \right)Y\|^2}{\|Y\|^2} \\
&= 1-\frac{\|\left(I - P_X\right)Y\|^2}{\|Y\|^2} \\
&= 1-\frac{Y^\top \left(I - P_X\right)^\top \left(I - P_X\right)Y}{\|Y\|^2} \\
&= 1-\frac{Y^\top \left(I - P_X - P_X^\top  + P_X^\top  P_X\right)Y}{\|Y\|^2} \\
&= \frac{Y^\top \left(P_X + P_X^\top  - P_X^\top  P_X\right)Y}{\|Y\|^2}
,\end{align*}
$$

[ja]: #
ここで $P_X= X(X^\top  X)^{-1}X^\top $ は射影行列と呼ばれ $P_X^2=P_X, P_X^\top =P_X$ が成立する重要な対称行列である (代入より簡単に証明できる。幾何学的には $X$ が張る部分空間へ $Y$ を投射する行列と解釈される)。
これとSVDによる $X,X^+$ の特異値分解と合わせて $\mathrm{R}^2$ は以下のとおり式変形される。

[en]: #
where $P_X= X(X^\top  X)^{-1}X^\top $ is called the projection matrix, an important symmetric matrix satisfying $P_X^2=P_X$ and $P_X^\top =P_X$ (this can be verified by direct substitution; geometrically, it projects $Y$ onto the subspace spanned by $X$).
Combining this with the SVD of $X$ and $X^+$, we can transform $\mathrm{R}^2$ as follows:

[END]: #
$$
\begin{align*}
\mathrm{R}^2&=\frac{Y^\top  P_X Y}{\|Y\|^2}\\
&=\frac{Y^\top  X(X^\top  X)^{-1}X^\top  Y}{\|Y\|^2}\\
&=\frac{Y^\top  X X^+ Y}{\|Y\|^2}\\
&=\frac{Y^\top  U\Sigma V^\top  V \Sigma^+ U^\top  Y}{\|Y\|^2}\\
&=\frac{Y^\top  U U^\top  Y}{\|Y\|^2}\\
&=\frac{\|U^\top  Y\|^2}{\|Y\|^2}
.\end{align*}
$$

[ja]: #
また三角不等式と直交性より $0 \leq \mathrm{R}^2\leq 1$ であると分かる ($P_X Y \perp (Y - P_X Y)$より$\|Y\|^2 = \|P_X Y\|^2+ \|Y - P_X Y\|^2$)。

長くなったが問題の説明に移ろう。
今行列 $X\in\mathbb{R}^{n\times p}, Y\in\mathbb{R}^{n}$ が与えられる。
ただし $\sum_{j=1}^{n} y_j=0$ となるように予め正規化されている。
上記の式を踏まえ $X$ の $Y$ に関する決定係数 $\mathrm{R}^2$ を求めるコードを書け。
ただし、前問同様`np.linalg.lstsq`は用いてはならず、また回答は `np.linalg.lstsq`を用いて検証される。

[en]: #
From the triangle inequality and orthogonality, we have $0 \leq \mathrm{R}^2\leq 1$ (since $P_X Y \perp (Y - P_X Y)$, we have $\|Y\|^2 = \|P_X Y\|^2+ \|Y - P_X Y\|^2$).

Now for the task.
Matrices $X\in\mathbb{R}^{n\times p}$ and $Y\in\mathbb{R}^{n}$ are given, where $Y$ is normalized such that $\sum_{j=1}^{n} y_j=0$.
Write code to compute the coefficient of determination $\mathrm{R}^2$ of $X$ with respect to $Y$.
As in the previous question, do not use `np.linalg.lstsq`; your answer will be validated using `np.linalg.lstsq`.

[END]: #
- $X$: `np.ndarray`
  - `shape`: `(n, p)`
  - `dtype`: `np.float64`
- $Y$: `np.ndarray`
  - `shape`: `(n,)`
  - `dtype`: `np.float64`
- $1 \leq p  \leq 10^2$
- $p \leq n \leq 10^{3}$
- Return: `float`

[tips]: #
- [`np.linalg.pinv`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.pinv.html)
- [`np.linalg.svd`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html)

[/tips]: #

In [ ]:
def solution(x, y):
    # BEGIN DO NOT USE `np.linalg.lstsq` for your answer.
    u, sigma, v = np.linalg.svd(x, full_matrices=False)
    uy = np.dot(u.T, y)
    return float(uy.dot(uy) / y.dot(y))

    # uy = x.T.dot(y)
    # uz = np.linalg.pinv(x).dot(y)
    # return float(uy.dot(uz) / y.dot(y))
    # END


test_func(solution, "05_04")
# show_solution("05_04")  # Uncomment it to see the solution.

Q5.5.

[ja]: #
デザイン行列 $X\in\mathbb{R}^{n\times p}$ が与えられる。
今分散共分散行列 $C=\mathrm{E}[(X-\mathrm{E}[X])^\top (X-\mathrm{E}[X])]$ $\left(\mathrm{E}[X]=\frac{1}{n}\sum\limits_{i=1}^{n-1}X_{ij}\right)$ に対してその固有値の絶対値を $(\lambda_i)_{0\leq i < p}$ とする。
この時 $X$ の[有効次元](https://arxiv.org/abs/0912.3832)は $N_\text{eff}$ は以下の式より定義される。

[en]: #
A design matrix $X\in\mathbb{R}^{n\times p}$ is given.
Let $C=\mathrm{E}[(X-\mathrm{E}[X])(X-\mathrm{E}[X])^\top ]$ $\left(\mathrm{E}[X]:=\frac{1}{n}\sum\limits_{i=1}^{n-1}X_{ij}\right)$ be the covariance matrix, and let $(\lambda_i)_{0\leq i < p}$ be the absolute values of its eigenvalues.
The [effective dimension](https://arxiv.org/abs/0912.3832) $N_\text{eff}$ of $X$ is defined by the following equation:

[END]: #
$$
\begin{align*}
N_\text{eff} &:= \frac{\left(\sum\limits_{i=0}^{p-1}\lambda_i\right)^2}{\sum\limits_{i=0}^{p-1}\lambda_i^2} \\
&=\left(\sum\limits_{i=0}^{p-1}\tilde{\lambda}_i^2\right)^{-1}
,\end{align*}
$$

[ja]: #
ただし $(\tilde{\lambda}_i)_{0\leq i < p}$ はその総和が1になるように各要素を一様に定数倍された固有値である。

有効次元 $N_\text{eff}$ を計算するコードを書け。

[en]: #
where $(\tilde{\lambda}_i)_{0\leq i < p}$ are the eigenvalues uniformly scaled so that their sum equals 1.

Write code to compute the effective dimension.

[END]: #
- $X$: `np.ndarray`
  - `shape`: `(n, p)`
  - `dtype`: `np.float64`
- $1 \leq p  \leq 10^2$
- $p \leq n \leq 10^{3}$
- Return: `float`

[tips]: #
- [`np.linalg.eig`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.eig.html)

[/tips]: #

In [ ]:
def solution(arr):
    # BEGIN
    arr = arr - arr.mean(axis=0, keepdims=True)
    eigs = np.linalg.eig(arr.T.dot(arr))[0]
    es = abs(eigs)
    es *= 1 / sum(es)
    return float(1 / (sum(es**2)))
    # END


test_func(solution, "05_05")
# show_solution("05_05")  # Uncomment it to see the solution.

[ja]: #
## 参考文献

[en]: #
## Reference

[ja]: #
[1] *NumPyの学び方*, NumPy team. https://numpy.org/ja/learn/

[2] Abbott, L. F., Rajan, K., & Sompolinsky, H. (2010). Interactions between Intrinsic and Stimulus-Evoked Activity in Recurrent Neural Networks (No. arXiv:0912.3832). arXiv. https://doi.org/10.48550/arXiv.0912.3832

[en]: #
[1] *NumPy - Learn*, NumPy team. https://numpy.org/learn/

[2] Abbott, L. F., Rajan, K., & Sompolinsky, H. (2010). Interactions between Intrinsic and Stimulus-Evoked Activity in Recurrent Neural Networks (No. arXiv:0912.3832). arXiv. https://doi.org/10.48550/arXiv.0912.3832